In [1]:
import pandas as pd
import numpy as np
import csv

In [2]:
def generateDataSet(csvFile):
    dataSet = []
    clusters = {}
    with open(csvFile, 'r') as csvfile:
        csvR = csv.reader(csvfile, delimiter = ',')
        for data in csvR:
            dataEntry = []
            for i in range(0, len(data)-1):
                dataEntry.append(float(data[i]))
            if(int(data[len(data)-1]) not in clusters):
                clusters[int(data[len(data)-1])] = []
                clusters[int(data[len(data)-1])].append(dataEntry)
            else:
                clusters[int(data[len(data)-1])].append(dataEntry)
            dataSet.append(dataEntry)
    return np.array(dataSet), clusters

dataSet, clusters = generateDataSet("glass.csv")

for i in clusters:
    print("Cluster Id is : ", i, "number  of points are ", len(clusters[i]))

Cluster Id is :  1 number  of points are  70
Cluster Id is :  2 number  of points are  76
Cluster Id is :  3 number  of points are  17
Cluster Id is :  5 number  of points are  13
Cluster Id is :  6 number  of points are  9
Cluster Id is :  7 number  of points are  29


MAHALNOBIS DISTANCE

In [3]:
def mahalnobisDistance(dataSet):
    distance = []
    meanXi = np.mean(dataSet, axis = 0)
    covX = np.cov(dataSet.astype(float), rowvar = False)
    invCovX = np.linalg.inv(covX)
    for Xi in dataSet:
        XiSubtractedMean = Xi - meanXi
        mul1 = np.dot(XiSubtractedMean, invCovX)
        mDistance = np.dot(mul1, XiSubtractedMean.T)
        distance.append(abs(mDistance)**0.5)
        # break
    return np.array(distance)
print(dataSet.shape)
mahalNobisDistances = mahalnobisDistance(dataSet)
print(np.mean(mahalNobisDistances))
print(mahalNobisDistances.shape)

(214, 9)
2.593694328961828
(214,)


LOF

In [4]:
class LOF:
    def __init__(self, dataSet, k):
        self.dataSet = dataSet
        self.k = k
        self.kThDistance = []
        self.kNearestNbrs = []
        self.Lrd = []
        self.Lof = []
    def calculateEuclidian(self, a, b):
        return np.sum(np.square(a-b))**(0.5)
    def calculateKthDistance(self):
        # calculating kth distance and kNN for every point
        for i in range(len(self.dataSet)):
            distanceArray = []
            for j in range(len(self.dataSet)):
                distanceArray.append(self.calculateEuclidian(i, j))
            # distanceArray = (np.sum((self.dataSet-self.dataSet[i])**2, axis = 1))**0.5
            indices = np.argsort(distanceArray)[1:self.k+1]
            temp2 = []
            for j in indices:
                temp2.append([distanceArray[j],j])
            self.kNearestNbrs.append(temp2)
            # print(temp2[-1])
            # self.kThDistance.append(distanceArray[indices[self.k-1]]) #because first will be the point itself.
            self.kThDistance.append(temp2[-1][0])
    def reachDistance(self, i, j):
        return max(self.calculateEuclidian(self.dataSet[i], self.dataSet[j]), self.kThDistance[j])
    def calculateLRD(self):
        for i in range(len(self.dataSet)):
            s = 0
            for point in self.kNearestNbrs[i]:
                s = s + self.reachDistance(i, point[1])
            s = s/self.k
            self.Lrd.append(1/s)
    def calculateLOF(self):
        for i in range(len(self.dataSet)):
            # print(i)
            lofOfPoint = 0
            for point in self.kNearestNbrs[i]:
                lofOfPoint = lofOfPoint + self.Lrd[point[1]]
            lofOfPoint = lofOfPoint/self.Lrd[i]
            lofOfPoint = lofOfPoint/self.k
            self.Lof.append(lofOfPoint)
        self.Lof = np.array(self.Lof)
    def algorithim(self):
        self.calculateKthDistance()
        self.calculateLRD()
        self.calculateLOF()
        # print(self.Lof)
tester = LOF(dataSet, 4)
tester.algorithim()
print(np.mean(tester.Lof))
print(np.max(tester.Lof))
print(tester.Lof.shape)

1.0199705256087903
1.9107859079361083
(214,)


Otsu Thresholding

In [6]:
from math import ceil, floor
class Otsu:
    def __init__(self, OutlierScores, noOfBins):
        self.OutlierScores = OutlierScores
        self.noOfBins = noOfBins
        self.threshHold = 0
        self.bins = {}

    def dividingLbins(self):
        indices = np.argsort(self.OutlierScores)
        maxValue = np.max(self.OutlierScores)
        minValue = np.min(self.OutlierScores)
        # self.bins = {}
        # for i in range(self.noOfBins+1):
        #     self.bins[i] = []
        # binWidth = (maxValue-minValue)/self.noOfBins
        # print(maxValue)
        # print(minValue)
        # print(binWidth)
        # print(binss)
        # print(binEdges)
        # for i in range(len(self.OutlierScores)):
        #     binNumber = floor((self.OutlierScores[i]-minValue)/binWidth) 
        #     self.bins[binNumber].append([self.OutlierScores[i], i]) 
        # for i in range(self.noOfBins):
        #     print("BinNumber is ", i, "number of points are ", len(self.bins[i]))   
    def calculatingT(self):
        minWVar = 1000000
        self.threshHold = 0
        # noOfOutliers  = 0
        binss, binEdges = np.histogram(self.OutlierScores, bins = self.noOfBins)
        # print(binss)
        # print(binEdges)
        ooo = 0
        minVar = 100000
        for t in range(1, self.noOfBins):
            Inliers = np.sum(binss[:t])
            Outliers = np.sum(binss[t:])
            Wo = Inliers/(Inliers+Outliers)
            W1 = Outliers/(Inliers+Outliers)
            WoMean = 0
            W1Mean = 0
            for i in range(t):
                WoMean = WoMean + binss[i]*(binEdges[i]+binEdges[i+1])/2
            for i in range(t, self.noOfBins):
                W1Mean = W1Mean + binss[i]*(binEdges[i]+binEdges[i+1])/2
            WoMean = WoMean/(Inliers)
            W1Mean= W1Mean/(Outliers)
            WoVar = W1Var = 0
            for i in range(t):
                d= (((binEdges[i]+binEdges[i+1])/2)-WoMean)**2 
                WoVar = WoVar + d*binss[i]
            for i in range(t, self.noOfBins):
                d= (((binEdges[i]+binEdges[i+1])/2)-W1Mean)**2 
                W1Var = W1Var + d*binss[i]
            WoVar = WoVar/(Inliers)
            W1Var = W1Var/(Outliers)
            var = Wo*WoVar + W1Var*W1
            if(var < minVar):
                minVar = var
                ooo = Outliers
                self.threshHold = (binEdges[t]+binEdges[t+1])/2
        print("Number of outliers are ",ooo)
        print("Threshold is ",self.threshHold)

    


            


otsuTester = Otsu(tester.Lof, 60)
otsuTester.dividingLbins()
otsuTester.calculatingT()

Number of outliers are  14
Threshold is  1.228697504564006
